# Día 4: Pydantic vs Dataclasses

## Descripción General

En Python moderno, tenemos dos herramientas principales para crear clases de datos: **dataclasses** (parte de la biblioteca estándar desde Python 3.7) y **Pydantic** (una biblioteca externa especializada en validación de datos). Ambas reducen el código boilerplate al crear clases que principalmente almacenan datos, pero tienen diferencias fundamentales en su propósito y capacidades.

En este notebook aprenderás cuándo usar cada una, sus ventajas y desventajas, y cómo elegir la herramienta correcta para tu caso de uso específico en proyectos de IA y ciencia de datos.

## Objetivos de Aprendizaje

Al finalizar este notebook, serás capaz de:

1. Comprender las diferencias fundamentales entre dataclasses y Pydantic
2. Crear clases de datos usando ambas herramientas con sintaxis correcta
3. Implementar validación de datos automática con Pydantic
4. Decidir cuándo usar dataclasses vs Pydantic según el contexto
5. Aplicar mejores prácticas de validación de datos en proyectos de IA

## Python Dataclasses: Lo Básico

### El Problema que Resuelven

Antes de Python 3.7, crear una clase simple para almacenar datos requería mucho código repetitivo (boilerplate):

```
Clase tradicional
├── __init__() - asignar cada atributo manualmente
├── __repr__() - representación en string
├── __eq__() - comparación de igualdad
└── Mucho código repetitivo
```

Las **dataclasses** automatizan la generación de estos métodos especiales.

In [ ]:
# BAD: Too much boilerplate for a simple data container
class UserOldStyle:
    """User data container - old style with boilerplate."""
    
    def __init__(self, name: str, age: int, email: str) -> None:
        self.name = name
        self.age = age
        self.email = email
    
    def __repr__(self) -> str:
        return f"UserOldStyle(name={self.name!r}, age={self.age!r}, email={self.email!r})"
    
    def __eq__(self, other: object) -> bool:
        if not isinstance(other, UserOldStyle):
            return NotImplemented
        return (self.name, self.age, self.email) == (other.name, other.age, other.email)

# Test the old style
user1 = UserOldStyle("Alice", 30, "alice@example.com")
print(user1)
print(f"Equals itself: {user1 == user1}")

In [ ]:
# GOOD: Dataclass with automatic method generation
from dataclasses import dataclass

@dataclass
class User:
    """User data container using dataclass."""
    name: str
    age: int
    email: str

# Test the dataclass - much cleaner!
user2 = User("Alice", 30, "alice@example.com")
print(user2)
print(f"Equals itself: {user2 == user2}")

# Dataclass automatically generates __init__, __repr__, __eq__
user3 = User("Alice", 30, "alice@example.com")
print(f"user2 == user3: {user2 == user3}")

### Aprendizaje Clave

**Dataclasses** reducen el código boilerplate al generar automáticamente `__init__()`, `__repr__()`, y `__eq__()`. Son perfectas para contenedores de datos simples sin necesidad de validación compleja.

**Referencia oficial:** [PEP 557 - Data Classes](https://peps.python.org/pep-0557/)

## Pydantic: Validación de Datos Automática

### El Problema que Resuelve

Las dataclasses no validan los datos automáticamente. Si pasas un tipo incorrecto, Python lo acepta sin quejarse:

```
Problema con dataclasses
├── No valida tipos en runtime
├── No convierte tipos automáticamente
├── No valida restricciones (ej: edad > 0)
└── Requiere validación manual
```

**Pydantic** resuelve esto con validación automática y conversión de tipos.

In [ ]:
# BAD: Dataclass accepts invalid data without validation
from dataclasses import dataclass

@dataclass
class UserDataclass:
    """User with dataclass - no runtime validation."""
    name: str
    age: int
    email: str

# This creates an invalid user - age is a string!
invalid_user = UserDataclass("Bob", "not a number", "bob@example.com")
print(f"Invalid user created: {invalid_user}")
print(f"Age type: {type(invalid_user.age)}")

# This will cause errors later in the code
try:
    next_year_age = invalid_user.age + 1  # TypeError!
except TypeError as e:
    print(f"Error: {e}")

In [ ]:
# GOOD: Pydantic validates and coerces types automatically
from pydantic import BaseModel, EmailStr, Field

class UserPydantic(BaseModel):
    """User with Pydantic - automatic validation."""
    name: str
    age: int = Field(gt=0, lt=150)  # age must be between 0 and 150
    email: str

# Pydantic converts string "25" to int 25
valid_user = UserPydantic(name="Bob", age="25", email="bob@example.com")
print(f"Valid user: {valid_user}")
print(f"Age type: {type(valid_user.age)}")
print(f"Age value: {valid_user.age}")

# Pydantic rejects invalid data
try:
    invalid_user = UserPydantic(name="Charlie", age="not a number", email="charlie@example.com")
except Exception as e:
    print(f"\nValidation error: {type(e).__name__}")
    print(f"Details: {e}")

### Aprendizaje Clave

**Pydantic** valida datos automáticamente en runtime, convierte tipos cuando es posible, y permite definir restricciones complejas. Es esencial para APIs, configuraciones, y procesamiento de datos externos.

**Referencia oficial:** [Pydantic Documentation](https://docs.pydantic.dev/)

## Comparación: Dataclasses vs Pydantic

| Característica | Dataclasses | Pydantic |
|----------------|-------------|----------|
| **Biblioteca** | Estándar (Python 3.7+) | Externa (pip install pydantic) |
| **Validación de tipos** | Solo type hints (no runtime) | Validación automática en runtime |
| **Conversión de tipos** | No | Sí (coerción automática) |
| **Validación personalizada** | Manual | Decoradores y Field() |
| **Serialización JSON** | Manual (con asdict()) | Automática (.model_dump(), .model_dump_json()) |
| **Performance** | Más rápido | Ligeramente más lento (por validación) |
| **Caso de uso** | Estructuras de datos internas | APIs, configuración, datos externos |
| **Curva de aprendizaje** | Muy simple | Moderada |

### Pregunta de Comprensión

¿Por qué Pydantic es más lento que dataclasses? ¿Vale la pena el costo?

In [ ]:
# Demonstrating Pydantic's type coercion
from pydantic import BaseModel
from typing import List

class DataPoint(BaseModel):
    """Data point for ML model."""
    feature_1: float
    feature_2: float
    label: int
    tags: List[str]

# Pydantic converts strings to appropriate types
data = DataPoint(
    feature_1="3.14",  # string -> float
    feature_2=2,        # int -> float
    label="1",          # string -> int
    tags=["train", "validated"]
)

print(f"Data: {data}")
print(f"feature_1 type: {type(data.feature_1)}")
print(f"feature_2 type: {type(data.feature_2)}")
print(f"label type: {type(data.label)}")

# This is extremely useful when reading data from JSON, CSV, or APIs

In [ ]:
# JSON serialization comparison
from dataclasses import dataclass, asdict
from pydantic import BaseModel
import json

@dataclass
class ConfigDataclass:
    """Configuration using dataclass."""
    model_name: str
    learning_rate: float
    epochs: int

class ConfigPydantic(BaseModel):
    """Configuration using Pydantic."""
    model_name: str
    learning_rate: float
    epochs: int

# Dataclass: manual conversion
config_dc = ConfigDataclass("bert-base", 0.001, 10)
config_dc_json = json.dumps(asdict(config_dc))
print(f"Dataclass JSON: {config_dc_json}")

# Pydantic: automatic conversion
config_pyd = ConfigPydantic(model_name="bert-base", learning_rate=0.001, epochs=10)
config_pyd_json = config_pyd.model_dump_json()
print(f"Pydantic JSON: {config_pyd_json}")

# Pydantic: easy parsing from JSON
json_str = '{"model_name": "gpt-2", "learning_rate": "0.002", "epochs": "20"}'
config_from_json = ConfigPydantic.model_validate_json(json_str)
print(f"\nParsed from JSON: {config_from_json}")
print(f"learning_rate type: {type(config_from_json.learning_rate)}")

## ¿Cuándo Usar Cada Una?

### Usa **Dataclasses** cuando:

1. ✅ Trabajas con estructuras de datos **internas** de tu aplicación
2. ✅ No necesitas validación de datos en runtime
3. ✅ Quieres **máxima performance** (sin overhead de validación)
4. ✅ Prefieres usar solo la **biblioteca estándar** (sin dependencias externas)
5. ✅ Los datos ya están validados o vienen de fuentes confiables

**Ejemplo:** Estructuras de datos intermedias en algoritmos, resultados de cálculos internos.

### Usa **Pydantic** cuando:

1. ✅ Recibes datos de **fuentes externas** (APIs, archivos, usuarios)
2. ✅ Necesitas **validación automática** de tipos y restricciones
3. ✅ Trabajas con **JSON** frecuentemente (APIs REST, configuraciones)
4. ✅ Quieres **conversión automática** de tipos (strings a números, etc.)
5. ✅ Necesitas **validaciones complejas** (rangos, formatos, dependencias entre campos)

**Ejemplo:** Modelos de API con FastAPI, configuraciones de aplicación, parseo de datos de ML.

### Aprendizaje Clave

La regla general: **Dataclasses para datos internos, Pydantic para datos externos**. Si recibes datos de fuera de tu control (usuarios, APIs, archivos), usa Pydantic. Si son estructuras internas que tú controlas, dataclasses es suficiente.

**Referencia oficial:** [Pydantic vs Dataclasses](https://docs.pydantic.dev/latest/why/)

## Ejercicios Prácticos

### Ejercicio 1: Convertir Dataclass a Pydantic (Básico)

Tienes una dataclass que representa un modelo de ML. Conviértela a Pydantic y añade validaciones:

- `model_name` no puede estar vacío
- `learning_rate` debe estar entre 0 y 1
- `batch_size` debe ser mayor que 0
- `epochs` debe estar entre 1 y 1000

In [ ]:
# TODO: Convert this dataclass to Pydantic with validations
from dataclasses import dataclass

@dataclass
class MLModelConfig:
    """ML model configuration."""
    model_name: str
    learning_rate: float
    batch_size: int
    epochs: int

# Your Pydantic version here:
# from pydantic import BaseModel, Field
#
# class MLModelConfig(BaseModel):
#     ...

# Test your implementation
# config = MLModelConfig(model_name="bert", learning_rate=0.001, batch_size=32, epochs=10)
# print(config)

### Ejercicio 2: Validación Personalizada (Intermedio)

Crea un modelo Pydantic para un dataset de ML que valide:

- `dataset_name`: string no vacío
- `num_samples`: entero positivo
- `num_features`: entero positivo
- `train_split`: float entre 0 y 1
- `test_split`: float entre 0 y 1
- **Validación adicional**: `train_split + test_split` debe ser <= 1.0

In [ ]:
# TODO: Create a Pydantic model with custom validation
from pydantic import BaseModel, Field, model_validator

# Your implementation here:
# class DatasetConfig(BaseModel):
#     dataset_name: str = Field(min_length=1)
#     num_samples: int = Field(gt=0)
#     num_features: int = Field(gt=0)
#     train_split: float = Field(ge=0, le=1)
#     test_split: float = Field(ge=0, le=1)
#     
#     @model_validator(mode='after')
#     def check_splits_sum(self):
#         # TODO: Validate that train_split + test_split <= 1.0
#         pass

# Test your implementation
# valid_config = DatasetConfig(
#     dataset_name="iris",
#     num_samples=150,
#     num_features=4,
#     train_split=0.7,
#     test_split=0.3
# )
# print(valid_config)

### Ejercicio 3: JSON Parsing con Pydantic (Avanzado)

Crea un modelo Pydantic que parsee configuraciones de experimentos de ML desde JSON. El JSON puede venir con tipos incorrectos (strings en lugar de números).

Requisitos:
- Parsear JSON con conversión automática de tipos
- Validar que todos los campos sean válidos
- Exportar de vuelta a JSON
- Manejar errores de validación apropiadamente

In [ ]:
# TODO: Create a Pydantic model for experiment configuration
from pydantic import BaseModel, Field
from typing import List
import json

# Example JSON (with string numbers)
experiment_json = '''
{
    "experiment_name": "bert_finetuning",
    "model_type": "transformer",
    "learning_rate": "0.0001",
    "batch_size": "16",
    "epochs": "5",
    "metrics": ["accuracy", "f1", "precision"]
}
'''

# Your Pydantic model here:
# class ExperimentConfig(BaseModel):
#     ...

# Parse and validate
# try:
#     config = ExperimentConfig.model_validate_json(experiment_json)
#     print(f"Parsed config: {config}")
#     print(f"\nAs JSON: {config.model_dump_json(indent=2)}")
# except Exception as e:
#     print(f"Validation error: {e}")

## Resumen

En este notebook hemos aprendido:

1. **Dataclasses** reducen el boilerplate al generar automáticamente `__init__()`, `__repr__()`, y `__eq__()`
2. **Pydantic** añade validación automática de tipos y conversión de datos en runtime
3. **Dataclasses** son ideales para estructuras de datos internas donde no necesitas validación
4. **Pydantic** es esencial para datos externos (APIs, archivos, configuraciones) que requieren validación
5. Pydantic facilita el trabajo con JSON mediante serialización y parsing automáticos

### Próximos Pasos

En el siguiente notebook exploraremos **Error Handling y Exceptions**, aprendiendo cómo manejar errores de validación y otros problemas de manera robusta en aplicaciones de IA.

## Preguntas de Autoevaluación

### 1. ¿Cuál es la principal diferencia entre dataclasses y Pydantic?

**Respuesta:** Dataclasses generan métodos especiales automáticamente pero no validan datos en runtime. Pydantic valida y convierte tipos automáticamente, lo que lo hace ideal para datos externos.

### 2. ¿Por qué Pydantic convierte `"25"` (string) a `25` (int) automáticamente?

**Respuesta:** Pydantic realiza coerción de tipos (type coercion) para facilitar el trabajo con datos externos como JSON, donde los números a menudo vienen como strings. Esto reduce el código de conversión manual.

### 3. ¿Cuándo deberías usar dataclasses en lugar de Pydantic?

**Respuesta:** Usa dataclasses para estructuras de datos internas donde no necesitas validación y quieres máxima performance. Son perfectas para resultados intermedios de cálculos o estructuras de datos que tú controlas completamente.

### 4. ¿Cómo defines una validación personalizada en Pydantic?

**Respuesta:** Puedes usar `Field()` para validaciones simples (rangos, longitudes) o `@model_validator` para validaciones complejas que involucran múltiples campos.

### 5. ¿Por qué Pydantic es más lento que dataclasses?

**Respuesta:** Pydantic realiza validación y conversión de tipos en runtime, lo que añade overhead computacional. Sin embargo, este costo es generalmente insignificante comparado con los beneficios de validación automática y prevención de errores.

### 6. ¿Qué ventaja tiene Pydantic para trabajar con APIs REST?

**Respuesta:** Pydantic facilita el parsing y serialización de JSON automáticamente con `model_validate_json()` y `model_dump_json()`, y valida que los datos recibidos cumplan con el esquema esperado.

### 7. ¿Puedes usar dataclasses y Pydantic juntos en el mismo proyecto?

**Respuesta:** Sí, es una práctica común. Usa Pydantic en los límites de tu aplicación (APIs, configuración) para validar datos externos, y dataclasses internamente para estructuras de datos donde no necesitas validación.

## Recursos y Referencias Oficiales

### Documentación Oficial

- **[PEP 557 - Data Classes](https://peps.python.org/pep-0557/)**: Propuesta oficial de Python para dataclasses
  - Explica el diseño y motivación detrás de dataclasses
  - Incluye ejemplos de uso y comparaciones con alternativas

- **[Python dataclasses Documentation](https://docs.python.org/3/library/dataclasses.html)**: Documentación oficial de la biblioteca estándar
  - Referencia completa de decoradores y funciones
  - Ejemplos de uso avanzado (frozen, order, etc.)

- **[Pydantic Documentation](https://docs.pydantic.dev/)**: Documentación oficial de Pydantic
  - Guías completas de validación y conversión de tipos
  - Ejemplos de uso con FastAPI y otras bibliotecas

- **[Pydantic - Why Pydantic?](https://docs.pydantic.dev/latest/why/)**: Comparación oficial con alternativas
  - Explica cuándo usar Pydantic vs dataclasses
  - Benchmarks de performance

### Estándares/PEPs

- **[PEP 484 - Type Hints](https://peps.python.org/pep-0484/)**: Base de los type hints en Python
  - Fundamental para entender cómo funcionan dataclasses y Pydantic

### Herramientas Relacionadas

- **[FastAPI](https://fastapi.tiangolo.com/)**: Framework web que usa Pydantic extensivamente
  - Ejemplos prácticos de Pydantic en APIs REST
  - Validación automática de requests y responses

- **[attrs](https://www.attrs.org/)**: Alternativa a dataclasses con más features
  - Precursor de dataclasses con funcionalidad adicional

### Mejores Prácticas

- **[Real Python - Data Classes](https://realpython.com/python-data-classes/)**: Tutorial completo sobre dataclasses
  - Ejemplos prácticos y casos de uso

- **[Real Python - Pydantic](https://realpython.com/python-pydantic/)**: Tutorial completo sobre Pydantic
  - Validación avanzada y casos de uso reales

### Notas Importantes

- Todos los enlaces están actualizados a partir de 2024
- Se recomienda revisar la documentación oficial regularmente
- Pydantic v2 (2023+) tiene cambios significativos respecto a v1